### Import necessary libraries

In [1]:
import gurobipy as gp
from gurobipy import GRB

### INPUT DATA (Generic dictionaries)

In [2]:
facilities = ['A', 'B', 'C', 'D', 'E']  #5 facility locations
demands = ['1', '2', '3', '4', '5', '6']  #6 demand locations
products = ['Alpha', 'Beta', 'Gamma']  #3 products
resources = ['Steel', 'Plastic', 'Energy', 'Labour']  #4 resources

In [3]:
# Input data
# Resource requirements for each product
resource_req = { 
    'Alpha': {'Steel': 2, 'Plastic': 3, 'Energy': 5, 'Labour': 4},
    'Beta': {'Steel': 4, 'Plastic': 1, 'Energy': 2, 'Labour': 3},
    'Gamma': {'Steel': 1, 'Plastic': 4, 'Energy': 3, 'Labour': 2}
}

In [4]:
# Resource prices per facility
resource_price = {  
    'A': {'Steel': 5.0, 'Plastic': 4.0, 'Energy': 3.5, 'Labour': 6.0},
    'B': {'Steel': 4.5, 'Plastic': 4.2, 'Energy': 3.8, 'Labour': 6.5},
    'C': {'Steel': 5.2, 'Plastic': 3.8, 'Energy': 3.6, 'Labour': 5.8},
    'D': {'Steel': 4.8, 'Plastic': 4.5, 'Energy': 4.0, 'Labour': 6.2},
    'E': {'Steel': 5.5, 'Plastic': 3.9, 'Energy': 3.7, 'Labour': 6.1}
}

In [5]:
# Production bounds
bounds = {  # (L_f, U_f) i.e the lower and upper bounds
    'A': (200, 800),
    'B': (150, 1000),
    'C': (50, 600),
    'D': (250, 1200),
    'E': (180, 900)
}

In [6]:
# The demand per location per product
demand_data = {  
    '1': {'Alpha': 205, 'Beta': 380, 'Gamma': 590},
    '2': {'Alpha': 475, 'Beta': 215, 'Gamma': 800},
    '3': {'Alpha': 600, 'Beta': 490, 'Gamma': 270},
    '4': {'Alpha': 900, 'Beta': 675, 'Gamma': 450},
    '5': {'Alpha': 350, 'Beta': 750, 'Gamma': 100},
    '6': {'Alpha': 80, 'Beta': 950, 'Gamma': 410}
}

In [7]:
# Selling price per location per product
price_data = {  
    '1': {'Alpha': 65, 'Beta': 50, 'Gamma': 42},
    '2': {'Alpha': 60, 'Beta': 45, 'Gamma': 55},
    '3': {'Alpha': 48, 'Beta': 70, 'Gamma': 38},
    '4': {'Alpha': 80, 'Beta': 65, 'Gamma': 42},
    '5': {'Alpha': 55, 'Beta': 30, 'Gamma': 25},
    '6': {'Alpha': 90, 'Beta': 85, 'Gamma': 40}
}

In [8]:
# Shipping price of each product
ship_cost = {'Alpha': 4, 'Beta': 3.5, 'Gamma': 2}  

In [9]:
dist = {  # dist_{f,d} (None = There is no connection between facility and demand location)
    'A': {'1': 120, '2': 30, '3': None, '4': 250, '5': None, '6': None},
    'B': {'1': 85, '2': 60, '3': 100, '4': 180, '5': None, '6': None},
    'C': {'1': None, '2': None, '3': 20, '4': 80, '5': 95, '6': None},
    'D': {'1': None, '2': 150, '3': None, '4': 160, '5': None, '6': 180},
    'E': {'1': None, '2': None, '3': None, '4': 140, '5': 175, '6': 200}
}

In [10]:
# Total budget given
budget = 500000

### Create the Model

In [11]:
m = gp.Model("Soltari_Logistics")
m.ModelSense = GRB.MAXIMIZE

Restricted license - for non-production use only - expires 2026-11-23


In [12]:
# Decision Variables
y = m.addVars(facilities, vtype=GRB.BINARY, name="y_open")
x = m.addVars(facilities, products, lb=0.0, name="x_prod")
ship_keys = [(f, d, p) for f in facilities for d in demands for p in products if dist[f][d] is not None]
z = m.addVars(ship_keys, lb=0.0, name="z_ship")

In [13]:
# Our Objective is to  maximize the revenue
# We need to sum up all the sales minus costs
arrived = {}  # This represents the total units of p arriving at d from all connected facilities
for d in demands:
    for p in products:
        arrived[(d, p)] = gp.quicksum(z[(f, d, p)] for f in facilities if dist[f][d] is not None)
        m.addConstr(arrived[(d, p)] <= demand_data[d][p], name=f"demand_{d}_{p}")
revenue = gp.quicksum(price_data[d][p] * arrived[(d, p)] for d in demands for p in products)
m.setObjective(revenue, GRB.MAXIMIZE)

In [14]:
# Constraints

# Production bounds (only if facility isopened)
for f in facilities:
    total_prod_f = gp.quicksum(x[(f, p)] for p in products)
    m.addConstr(total_prod_f >= bounds[f][0] * y[f], name=f"min_prod_{f}")
    m.addConstr(total_prod_f <= bounds[f][1] * y[f], name=f"max_prod_{f}")

In [15]:
# Flow balance: produced = shipped
for f in facilities:
    for p in products:
        shipped = gp.quicksum(z[(f, d, p)] for d in demands if dist[f][d] is not None)
        m.addConstr(x[(f, p)] == shipped, name=f"flow_{f}_{p}")

In [16]:
# Budget constraint - total cost can't exceed budget
resource_cost = gp.quicksum(
    resource_price[f][r] * gp.quicksum(resource_req[p][r] * x[(f, p)] for p in products)
    for f in facilities for r in resources
)
shipping_cost = gp.quicksum(
    dist[f][d] * z[(f, d, p)] * ship_cost[p] for (f, d, p) in ship_keys
)
total_cost = resource_cost + shipping_cost
m.addConstr(total_cost <= budget, name="budget")

<gurobi.Constr *Awaiting Model Update*>

In [17]:
#Optimize

m.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: AMD Ryzen 7 7730U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 44 rows, 68 columns and 214 nonzeros
Model fingerprint: 0xfabbcf18
Variable types: 63 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [3e+01, 9e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+01, 5e+05]
Found heuristic solution: objective -0.0000000
Presolve removed 13 rows and 13 columns
Presolve time: 0.00s
Presolved: 31 rows, 55 columns, 189 nonzeros
Variable types: 50 continuous, 5 integer (5 binary)

Root relaxation: objective 1.495077e+05, 35 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0  

### Output

In [18]:
# Sanity check: Theoretical max revenue (ignores costs/constraints)
theoretical_max = sum(demand_data[d][p] * price_data[d][p] for d in demands for p in products)

In [19]:
# Output results (formatted for readability)
print("=== SOLTARI LOGISTICS OPTIMIZATION ===")
print(f"Status: {m.Status}")
if m.Status == GRB.OPTIMAL:
    print(f"Optimal Revenue: {m.ObjVal:.2f}")
    print(f"Theoretical Max Revenue: {theoretical_max:.2f}")
    print(f"Total Cost Used: {total_cost.getValue():.2f} (Budget: {budget})")
else:
    print("No optimal solution found.")
print()

print("=== Opened Facilities ===")
opened = [f for f in facilities if y[f].X > 0.5]
print(", ".join(opened) if opened else "None")
print()

print("=== Production (units per facility/product) ===")
for f in facilities:
    if y[f].X > 0.5:
        total_f = sum(x[(f, p)].X for p in products)
        print(f"Facility {f} (Total: {total_f:.2f}):")
        for p in products:
            val = x[(f, p)].X
            if val > 1e-3:
                print(f"  {p}: {val:.2f}")
        print()
print()

print("=== Shipments to Demand Locations (total units received per location/product) ===")
for d in demands:
    print(f"Demand {d}:")
    for p in products:
        total_to_d = sum(z[(f, d, p)].X for f in facilities if dist[f][d] is not None)
        if total_to_d > 1e-3:
            print(f"  {p}: {total_to_d:.2f} (Demand: {demand_data[d][p]}, Price: {price_data[d][p]})")
    print()
print()

print("=== Detailed Shipments (from facility to location/product) ===")
for f in opened:
    print(f"From {f}:")
    for d in demands:
        if dist[f][d] is not None:
            for p in products:
                val = z[(f, d, p)].X
                if val > 1e-3:
                    cost = dist[f][d] * val * ship_cost[p]
                    print(f"  To {d} {p}: {val:.2f} units (Ship cost: {cost:.2f})")
    print()

=== SOLTARI LOGISTICS OPTIMIZATION ===
Status: 2
Optimal Revenue: 149507.67
Theoretical Max Revenue: 496015.00
Total Cost Used: 500000.00 (Budget: 500000)

=== Opened Facilities ===
A, B, C, E

=== Production (units per facility/product) ===
Facility A (Total: 800.00):
  Alpha: 475.00
  Beta: 215.00
  Gamma: 110.00

Facility B (Total: 1000.00):
  Gamma: 1000.00

Facility C (Total: 600.00):
  Alpha: 110.00
  Beta: 490.00

Facility E (Total: 266.97):
  Gamma: 266.97


=== Shipments to Demand Locations (total units received per location/product) ===
Demand 1:
  Gamma: 310.00 (Demand: 590, Price: 42)

Demand 2:
  Alpha: 475.00 (Demand: 475, Price: 60)
  Beta: 215.00 (Demand: 215, Price: 45)
  Gamma: 800.00 (Demand: 800, Price: 55)

Demand 3:
  Beta: 490.00 (Demand: 490, Price: 70)

Demand 4:
  Alpha: 110.00 (Demand: 900, Price: 80)
  Gamma: 266.97 (Demand: 450, Price: 42)

Demand 5:

Demand 6:


=== Detailed Shipments (from facility to location/product) ===
From A:
  To 2 Alpha: 475.00 uni